### This model intends to find the maginitude of events

#### Steps
1. Web Crawl News about COVID19 ( requires pretagging )
2. Data Cleaning
3. EDA ( HotWord, tf-idf)
4. Vectorization (word2vec, bert)
5. Classification (GRU, bert)


A lot of the tagging requires manual work, which i've been trying to do, to get the model to recognize the right word

In [27]:
import pandas as pd
import numpy as np

In [28]:

API_COLUMNS = ["Title", "Description", "NewsURL", "PublishedTime", "SourceName" , "SourceURL"]

api_content = pd.read_csv("../Data/NewsContent.csv")

In [29]:
api_content

,Title,Description,NewsURL,PublishedTime,SourceName,SourceURL
0,TSA issues new directive for anyone entering t...,issued a new directive effective Sunday requir...,https://www.msn.com/en-us/news/us/tsa-issues-n...,2020-02-01,CNN on MSN.com,https://www.msn.com
1,"As coronavirus spreads, TSA issues new directi...",issued a new directive effective Sunday requir...,https://edition.cnn.com/2020/02/02/politics/ts...,2020-02-01,CNN,https://edition.cnn.com
2,Neenah family secures seats on flight from cor...,"NEENAH — Daisy, Abigail, and Adalynn Roth have...",https://fox6now.com/2020/02/02/neenah-family-s...,2020-02-01,FOX6 Milwaukee,https://fox6now.com
3,"US companies suspend China operations, restric...","Disney, Tesla, airlines and other companies wi...",https://www.cnbc.com/2020/02/01/coronavirus-co...,2020-02-01,CNBC,https://www.cnbc.com
4,Coronavirus: US confirms eighth case with quar...,Federal authorities say Massachusetts patient ...,https://www.theguardian.com/world/2020/feb/01/...,2020-02-01,The Guardian,https://www.theguardian.com
...,...,...,...,...,...,...
1187,Take-Two CEO: Hurricane Sandy helped us prepar...,Two CEO Strauss Zelnick discusses how the game...,https://www.digitaltrends.com/gaming/take-two-...,2020-06-04,Digital Trends,https://www.digitaltrends.com
1188,Take-Two CEO: Hurricane Sandy helped us prepar...,Two CEO Strauss Zelnick discusses how the game...,https://www.digitaltrends.com/gaming/take-two-...,2020-06-04,Digital Trends,https://www.digitaltrends.com
1189,Take-Two CEO: Hurricane Sandy helped us prepar...,Two CEO Strauss Zelnick discusses how the game...,https://www.digitaltrends.com/gaming/take-two-...,2020-06-04,Digital Trends,https://www.digitaltrends.com
1190,Take-Two CEO: Hurricane Sandy helped us prepar...,Two CEO Strauss Zelnick discusses how the game...,https://www.digitaltrends.com/gaming/take-two-...,2020-06-04,Digital Trends,https://www.digitaltrends.com


In [30]:
api_content.head()

,Title,Description,NewsURL,PublishedTime,SourceName,SourceURL
0,TSA issues new directive for anyone entering t...,issued a new directive effective Sunday requir...,https://www.msn.com/en-us/news/us/tsa-issues-n...,2020-02-01,CNN on MSN.com,https://www.msn.com
1,"As coronavirus spreads, TSA issues new directi...",issued a new directive effective Sunday requir...,https://edition.cnn.com/2020/02/02/politics/ts...,2020-02-01,CNN,https://edition.cnn.com
2,Neenah family secures seats on flight from cor...,"NEENAH — Daisy, Abigail, and Adalynn Roth have...",https://fox6now.com/2020/02/02/neenah-family-s...,2020-02-01,FOX6 Milwaukee,https://fox6now.com
3,"US companies suspend China operations, restric...","Disney, Tesla, airlines and other companies wi...",https://www.cnbc.com/2020/02/01/coronavirus-co...,2020-02-01,CNBC,https://www.cnbc.com
4,Coronavirus: US confirms eighth case with quar...,Federal authorities say Massachusetts patient ...,https://www.theguardian.com/world/2020/feb/01/...,2020-02-01,The Guardian,https://www.theguardian.com


In [37]:
temp = api_content.groupby(["PublishedTime", "SourceName"])

In [38]:
temp

In [46]:
temp.get_group(('2020-02-01', 'ABC'))

,Title,Description,NewsURL,PublishedTime,SourceName,SourceURL
6,US declares public health emergency over coron...,There are now eight confirmed cases in the U.S...,https://abcnews.go.com/Health/delta-suspends-u...,2020-02-01,ABC,https://abcnews.go.com


In [40]:
temp.groups

{('2020-02-01', 'ABC'): Int64Index([6], dtype='int64'),
 ('2020-02-01', 'BBC'): Int64Index([8], dtype='int64'),
 ('2020-02-01', 'CNBC'): Int64Index([3], dtype='int64'),
 ('2020-02-01', 'CNN'): Int64Index([1], dtype='int64'),
 ('2020-02-01', 'CNN on MSN.com'): Int64Index([0], dtype='int64'),
 ('2020-02-01', 'FOX6 Milwaukee'): Int64Index([2], dtype='int64'),
 ('2020-02-01', 'Fox News'): Int64Index([7], dtype='int64'),
 ('2020-02-01', 'The Guardian'): Int64Index([4], dtype='int64'),
 ('2020-02-01', 'The Hill'): Int64Index([5], dtype='int64'),
 ('2020-02-02', 'CNN on MSN.com'): Int64Index([10], dtype='int64'),
 ('2020-02-02', 'FXStreet'): Int64Index([9], dtype='int64'),
 ('2020-02-03', 'CNBC'): Int64Index([15], dtype='int64'),
 ('2020-02-03', 'Fox News'): Int64Index([12, 19], dtype='int64'),
 ('2020-02-03', 'Hartford Courant'): Int64Index([17], dtype='int64'),
 ('2020-02-03', 'SFGate'): Int64Index([13], dtype='int64'),
 ('2020-02-03', 'South China Morning Post'): Int64Index([11], dtype='in